# Seq2Seq Reference Code

1) 원래 코드에 주석 달기  
2) 새로운 데이터셋 적용하여 돌려보기  
3) 구현 시 주의해야할 사항 미리 생각해오기  

### Initial settings

In [1]:
import random
import math
import time
import spacy

import numpy as np

from tqdm.notebook import tqdm
from nltk.tokenize import word_tokenize

import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

In [2]:
SEED = 1234
BATCH_SIZE = 64

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### load dataset  
first, download the data from spacy
```
python -m spacy download en_core_web_sm
python -m spacy download fr_core_news_sm
```

In [3]:
spacy_fr = spacy.load('fr_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

In [15]:
def tokenize_fr_src(text):
    return [tok.text for tok in spacy_fr.tokenizer(text)][::-1]
def tokenize_fr_trg(text):
    return [tok.text for tok in spacy_fr.tokenizer(text)]
def tokenize_fr_src_nltk(text):
    return word_tokenize(text, language='french')[::-1]
def tokenize_fr_trg_nltk(text):
    return word_tokenize(text, language='french')

def tokenize_en_src(text):
    return [tok.text for tok in spacy_en.tokenizer(text)][::-1]
def tokenize_en_trg(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]
def tokenize_en_src_nltk(text):
    return word_tokenize(text, language='english')[::-1]
def tokenize_en_trg_nltk(text):
    return word_tokenize(text, language='english')

In [17]:
SRC_FR = Field(tokenize = tokenize_fr_src,
               init_token = '<sos>',
               eos_token = '<eos>', 
               lower = True)

TRG_EN = Field(tokenize = tokenize_en_trg,
               init_token = '<sos>',
               eos_token = '<eos>',
               lower = True)

### Load Dataset
French (.fr) data are not automatically downloadable.  
manually input train.fr and val.fr in ./data/multi30k directory

In [20]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.fr', '.en'), fields = (SRC_FR, TRG_EN))

In [54]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000


In [52]:
def print_pair_examples(dataset, printing_size=1):
    def numeric2ordinal(idx):
        if idx == 0:
            suff = 'st'
        elif idx == 1:
            suff = 'nd'
        elif idx == 2:
            suff = 'rd'
        else:
            suff = 'th'
        return str(idx+1) + suff
    data_size = len(dataset.examples)
    for _ in range(printing_size):
        rand_idx = random.randint(0, data_size)
        text_dict = vars(dataset.examples[rand_idx])
        print(numeric2ordinal(rand_idx), 'example:')
        print('source txt:', ' '.join(text_dict['src'][::-1]))
        print('target txt:', ' '.join(text_dict['trg']))    

In [53]:
print_pair_examples(train_data, 3)

7760th example:
source txt: de nombreux chiens noirs courent sur une pelouse .
target txt: many black dogs run in a grassy area .
572th example:
source txt: cinq personnes assises autour d' une table , à l' extérieur , jouent à un jeu de cartes .
target txt: five people are sitting around a table , outside , playing a card game .
1010th example:
source txt: un jeune garçon en t - shirt bleu et chapeau vert gonfle un ballon orange .
target txt: a young boy in a blue shirt and green hat blows up an orange balloon .


In [63]:
print_pair_examples(valid_data, 3)

933th example:
source txt: un jeune homme fait du skateboard sur un mur de parpaings .
target txt: a young man is skateboarding on a cement block wall .
966th example:
source txt: un homme portant des bottes d' équitation et un casque montant sur un cheval blanc qui saute un obstacle .
target txt: a man wearing riding boots and a helmet is riding a white horse , and the horse is jumping a hurdle .
962th example:
source txt: un cycliste portant un maillot jaune réalise un tour incroyable dans l' air .
target txt: a biker wearing a yellow shirt pulls of an incredible trick in the air .


In [64]:
print_pair_examples(test_data, 3)

827th example:
source txt: deux jeunes filles sont assises dans la rue , mangeant du maïs .
target txt: two young girls are sitting on the street eating corn .
597th example:
source txt: un groupe de personnes jouent sur un objet .
target txt: a group of people are playing on an object .
36th example:
source txt: un employé donne un sac à une femme tandis qu' elle regarde du poisson sur de la glace sur un marché de rue .
target txt: an employee is handing a woman a bag while she is browsing through fish on ice at a street market .


In [58]:
SRC_FR.build_vocab(train_data, min_freq = 2)
TRG_EN.build_vocab(train_data, min_freq = 2)
print(f"Unique tokens in source (fr) vocabulary: {len(SRC_FR.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG_EN.vocab)}")

Unique tokens in source (fr) vocabulary: 6470
Unique tokens in target (en) vocabulary: 5893


In [65]:
train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train_data, valid_data, test_data), 
                                                                      batch_size = BATCH_SIZE, device = device)

### seq2seq model


In [68]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.hid_dim = hid_dim
        self.n_layers = n_layers        
        self.embedding = nn.Embedding(input_dim, emb_dim)        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        #src = [src len, batch size]
        embedded = self.dropout(self.embedding(src))  #embedded = [src len, batch size, emb dim]
        outputs, (hidden, cell) = self.rnn(embedded)  #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        #outputs are always from the top hidden layer
        return hidden, cell

In [67]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers        
        self.embedding = nn.Embedding(output_dim, emb_dim)        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)        
        self.fc_out = nn.Linear(hid_dim, output_dim)        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):        
        #input = [batch size]
        
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)        
        #input = [1, batch size]        
        embedded = self.dropout(self.embedding(input))  # embedded = [1, batch size, emb dim]                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))        
        #prediction = [batch size, output dim]
        return prediction, hidden, cell

In [66]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device        
        assert encoder.hid_dim == decoder.hid_dim, "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        trg_len, batch_size = trg.shape
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1)             
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1        
        return outputs

In [69]:
INPUT_DIM = len(SRC_FR.vocab)
OUTPUT_DIM = len(TRG_EN.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [70]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(6470, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [71]:
def count_parameters(model):
    param_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f'The model has {param_count:,} trainable parameters')
count_parameters(model)

The model has 13,544,453 trainable parameters


In [72]:
optimizer = optim.Adam(model.parameters())

In [74]:
TRG_PAD_IDX = TRG_EN.vocab.stoi[TRG_EN.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [81]:
def train(model, iterator, optimizer, criterion, clip):    
    model.train()    
    epoch_loss = 0    
    for i, batch in enumerate(tqdm(iterator)):        
        src = batch.src
        trg = batch.trg  #trg = [trg len, batch size]
        optimizer.zero_grad()        
        output = model(src, trg)  #output = [trg len, batch size, output dim]        
        output_dim = output.shape[-1]        
        output = output[1:].view(-1, output_dim)  #output = [(trg len - 1) * batch size, output dim]
        trg = trg[1:].view(-1)  # trg = [(trg len - 1) * batch size]
        
        loss = criterion(output, trg)
        loss.backward()        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)        
        optimizer.step()        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [82]:
def evaluate(model, iterator, criterion):    
    model.eval()    
    epoch_loss = 0    
    with torch.no_grad():    
        for i, batch in enumerate(tqdm(iterator)):
            src = batch.src
            trg = batch.trg  #trg = [trg len, batch size]
            output = model(src, trg, 0) # output = [trg len, batch size, output dim]
            #turn off teacher forcing 
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)  # output = [(trg len - 1) * batch size, output dim]
            trg = trg[1:].view(-1)  #trg = [(trg len - 1) * batch size]
            loss = criterion(output, trg)            
            epoch_loss += loss.item()        
    return epoch_loss / len(iterator)

In [83]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [84]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):    
    start_time = time.time()
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')


Epoch: 01 | Time: 14m 44s
	Train Loss: 4.849 | Train PPL: 127.602
	 Val. Loss: 4.882 |  Val. PPL: 131.903



Epoch: 02 | Time: 14m 53s
	Train Loss: 4.358 | Train PPL:  78.072
	 Val. Loss: 4.698 |  Val. PPL: 109.768



Epoch: 03 | Time: 14m 44s
	Train Loss: 4.081 | Train PPL:  59.182
	 Val. Loss: 4.514 |  Val. PPL:  91.314



Epoch: 04 | Time: 14m 35s
	Train Loss: 3.845 | Train PPL:  46.754
	 Val. Loss: 4.416 |  Val. PPL:  82.739



Epoch: 05 | Time: 14m 41s
	Train Loss: 3.682 | Train PPL:  39.706
	 Val. Loss: 4.159 |  Val. PPL:  64.001



Epoch: 06 | Time: 15m 16s
	Train Loss: 3.541 | Train PPL:  34.487
	 Val. Loss: 4.129 |  Val. PPL:  62.110



Epoch: 07 | Time: 13m 52s
	Train Loss: 3.383 | Train PPL:  29.451
	 Val. Loss: 3.975 |  Val. PPL:  53.267



Epoch: 08 | Time: 13m 8s
	Train Loss: 3.218 | Train PPL:  24.978
	 Val. Loss: 3.828 |  Val. PPL:  45.955



Epoch: 09 | Time: 12m 57s
	Train Loss: 3.080 | Train PPL:  21.752
	 Val. Loss: 3.808 |  Val. PPL:  45.070



Epoch: 10 | Time: 13m 6s
	Train Loss: 2.948 | Train PPL:  19.069
	 Val. Loss: 3.772 |  Val. PPL:  43.464
